### Hyperparameter GridSearchCV LSTM-RNN Percobaan 2

- Variabel Terikat : Titik Panas
- Variabel Bebas : Curah Hujan

In [1]:
# pustaka untuk manipulasi data
import pandas as pd
from pandas import concat
from pandas import DataFrame
from pandas import read_csv
from pandas import read_excel
import numpy as np
from numpy import concatenate

# pustaka untuk waktu komputasi
import time
from datetime import datetime

# Pustaka untuk visualisasi data
import seaborn as sns # Visualization
from matplotlib import pyplot
from matplotlib import pyplot as plt

# Pustaka untuk membuat data latih dan data uji.
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Pustaka untuk membuat model prediksi LSTM-RNN
import itertools
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import Adam, Adamax, RMSprop, SGD

# Early stoping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Pustaka untuk  evaluasi model prediksi
from math import sqrt
from sklearn.metrics import mean_squared_error

In [2]:
# Set waktu komputasi
start = time.time()

In [3]:
# fix random seed for reproducibility
np.random.seed(1234)

In [4]:
# membaca dataset
dataset = read_excel("dataset/dataset.xlsx")

In [5]:
# set index tanggal
dataset = dataset.set_index("tanggal")

In [6]:
# set informasi dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2001-01-31 to 2020-12-31
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sst             240 non-null    float64
 1   soi             240 non-null    float64
 2   oni             240 non-null    float64
 3   curah_hujan     240 non-null    float64
 4   hotspot_sumsel  240 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 11.2 KB


In [7]:
# menampilkan dataset
print(dataset.head())

             sst  soi   oni  curah_hujan  hotspot_sumsel
tanggal                                                 
2001-01-31 -0.76  1.0 -0.68   418.600006               3
2001-02-28 -0.56  1.7 -0.52   295.399994               5
2001-03-31 -0.37  0.9 -0.44   360.600006              10
2001-04-30 -0.56  0.2 -0.34   403.700012              20
2001-05-31 -0.46 -0.5 -0.25   116.900002              39


### Studi kasus wilayah Sumatra Selatan

In [8]:
# memilih area studi
df_sumsel = dataset[["hotspot_sumsel", "curah_hujan"]]
df_sumsel.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2001-01-31 to 2020-12-31
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hotspot_sumsel  240 non-null    int64  
 1   curah_hujan     240 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 5.6 KB


In [9]:
print(df_sumsel.head())

            hotspot_sumsel  curah_hujan
tanggal                                
2001-01-31               3   418.600006
2001-02-28               5   295.399994
2001-03-31              10   360.600006
2001-04-30              20   403.700012
2001-05-31              39   116.900002


In [10]:
# ensure all data is float
values = df_sumsel.values
values = values.astype('float32')

### Normalisasi Data

In [11]:
# membuat fungsi max-min scaler
# menggunakan pustaka scikit-learn
def normalisasi_max_min(df):
    # memanggil fungsi max min scaler
    hasil = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    
    # proses max min scaler
    hasil = hasil.fit_transform(df)
    
    # pengembalian nilai
    return hasil

In [12]:
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

In [13]:
np.round(scaled[:5],6)

array([[-0.999723,  0.549573],
       [-0.999445,  0.082464],
       [-0.998751,  0.329668],
       [-0.997364,  0.493081],
       [-0.994728, -0.594313]], dtype=float32)

### Normalisasi dan Supervised learning

In [14]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    # return value
    return agg

In [15]:
# supervised learning
reframed = series_to_supervised(scaled, 1, 1)

In [16]:
# drop columns we don't want to predict
reframed.drop(reframed.columns[[3]], axis=1, inplace=True)

In [17]:
values = reframed.values

In [18]:
print(reframed.head())

   var1(t-1)  var2(t-1)   var1(t)
1  -0.999723   0.549573 -0.999445
2  -0.999445   0.082464 -0.998751
3  -0.998751   0.329668 -0.997364
4  -0.997364   0.493081 -0.994728
5  -0.994728  -0.594313 -0.999029


### Data latih dan Data uji

In [19]:
# split into train and test sets
train_size = int(len(values) * 0.8)
test_size = len(values) - train_size
train, test = values[0:train_size,:], values[train_size:len(values),:]

In [20]:
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [21]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [22]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(191, 1, 2) (191,) (48, 1, 2) (48,)


### Hyperparameter GridSearchCV LTSM-RNN

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
parameters = {'neurons' : [10],
              'activation' : ['sigmoid', 'tanh', 'relu', 'selu', 'elu', 'softplus'],
              'optimizer' : ['adam', 'adamax', 'rmsprop', 'sgd'],
              'dropout_rate' : [0.05, 0.1, 0.15, 0.2, 0.25],
              'epochs' : [2000],
              'batch_size' : [4, 8, 16, 32, 64],
              'verbose' : [0]}

In [ ]:
keys = parameters.keys()
values = (parameters[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]
print(len(combinations), str('kombinasi hyperparameter'))

In [ ]:
def build_classifier(neurons='', activation='', optimizer='', dropout_rate=''):
    tf.keras.backend.clear_session()
    
    # design network
    grid_model = Sequential()
    grid_model.add(LSTM(units=neurons, activation=activation, input_shape=(train_X.shape[1], train_X.shape[2])))
    grid_model.add(Dropout(dropout_rate))
    grid_model.add(Dense(1))

    # model compile
    grid_model.compile(loss='mae', optimizer=optimizer)
    
    # return value
    return grid_model

In [ ]:
grid_model = KerasRegressor(build_fn=build_classifier)

In [ ]:
grid_search = GridSearchCV(estimator=grid_model, param_grid=parameters, n_jobs=-1, cv=2, scoring= 'neg_root_mean_squared_error')

In [ ]:
grid_result = grid_search.fit(train_X, train_y)

In [ ]:
# summarize results
print("Best parameters: %f using %s\n" % (grid_search.best_score_, grid_search.best_params_))

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
result = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),pd.DataFrame(grid_search.cv_results_["mean_test_score"], columns=["score"])],axis=1)

In [ ]:
result.sort_values(by="score", ascending=False)

In [ ]:
result.sort_values(by="score", ascending=False).to_csv('gridsearch_percobaan2.csv',index=False)

### Evaluasi Model LSTM-RNN

In [ ]:
# Set akhir waktu komputasi 
end = time.time()

In [ ]:
# Proses menghitung waktu komputasi
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)

In [ ]:
# Hasil waktu komputasi
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))